In [ ]:
import pandas as pd
import numpy as np
from timedelta import Timedelta 

from lumibot.brokers import Alpaca
from lumibot.backtesting import BacktestingBroker, YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from lumibot.entities import Asset
from datetime import datetime 
from alpaca_trade_api import REST 

from src.get_data import GetUpdateData
from src.get_data import GetDataFRED
from ta import add_all_ta_features

from joblib import dump, load

import src.model_train
from imp import reload
reload(src.model_train)

import src.ml_algo_trade
from imp import reload
reload(src.ml_algo_trade)

C:\Users\18327\AppData\Local\Temp\ipykernel_19992\3167153815.py:20: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


<module 'src.ml_algo_trade' from 'c:\\Users\\18327\\Desktop\\stock_prediction\\src\\ml_algo_trade.py'>

In [ ]:
import json

# Load secrets from secrets.json
with open('secrets.json') as f:
    secrets = json.load(f)
    
api_key_twelve = secrets['default']['twelve_data_api']
api_key_fred = secrets['default']['fred_api']

api_key_alpaca = secrets['default']['alpaca_api_key']
api_secret_alpaca = secrets['default']['alpaca_api_secret']

In [ ]:
clf_model = load("models/candidate_models/lgbm-direction-classifier-2024-05-16-22-47-42.joblib")

# BackTesting

In [ ]:
data_raw = pd.read_csv('data/data_model.csv', index_col=0, parse_dates=True)
data = data_raw.copy()
drop_cols = ['Security', 'ticker', 'Headquarters Location', 'Date added', 'CIK', 'Founded', 'target_percent_gain', 'target_direction']
features = [c for c in data.columns if c not in drop_cols]

class MyStrategy(Strategy):
    def initialize(self, symbol: str = "SPY", cash_at_risk: float = 1, clf_model=clf_model, budget=1000):
        self.symbol = symbol
        self.clf_model = clf_model
        self.budget = budget
        self.sleeptime = "7D"
        self.last_trade = None
        # self.set_market("24/7")
        self.cash_at_risk = cash_at_risk
        self.df_ohlci = self.get_data()
        
        self.trading_iteration = 0

    def get_data(self):

        ticker = self.symbol.replace("-", "/")
        df_stock = (
            GetUpdateData(api_key=api_key_twelve)
            .get_data_single_ticker(ticker)
            .sort_values("datetime", ascending=True)
        )

        _, df_tickers = GetUpdateData(
            update_existing=False, api_key=api_key_twelve
        ).get_tickers()

        df_stock = df_stock.merge(
            df_tickers[["ticker", "GICS Sector", "GICS Sub-Industry"]],
            on="ticker",
            how="left",
        ).drop('ticker', axis=1)
        
        df_fred = GetDataFRED(api_key=api_key_fred).get_data()
        df_fred["datetime"] = pd.to_datetime(df_fred["datetime"])

        df_ohlci = add_all_ta_features(
            df_stock,
            open="open",
            high="high",
            low="low",
            close="close",
            volume="volume",
            fillna=True,
        )

        df_ohlci["datetime"] = pd.to_datetime(df_ohlci["datetime"])
        df_ohlci["month"] = df_ohlci["datetime"].dt.month
        df_ohlci["dayofweek"] = df_ohlci["datetime"].dt.weekday
        df_ohlci = df_ohlci.merge(df_fred, on="datetime", how="left")

        return df_ohlci

    def get_date(self):
        today = self.get_datetime()
        print(f'Today is {today}')
        today_str = today.strftime('%Y-%m-%d')
        yesterday = today - Timedelta(days=1)
        yesterday_str = yesterday.strftime('%Y-%m-%d')
        month = today.strftime('%m')
        day = today.strftime('%d')
        return today_str, yesterday_str
    
    def train_model(self):
        _, yesterday_str = self.get_date()
        
        # Retrain model on every 30 iterations
        if (self.trading_iteration % 200 == 0):
            print(f'Retraining data...')
            X_train, X_test, y_train, y_test = src.model_train.ModelPrepData(
                df=data, features=features, target="target_direction", test_start_date=yesterday_str, test_size=0.2
                ).create_train_test()
        
            self.clf_model = src.model_train.CreateModel(task='classification', model_type='LGBM', X_train=X_train, y_train=y_train).fit()


    def get_signal(self):
        
        # Need to use index lookup to overcome weekends with no data
        X = self.df_ohlci.copy()
        today_str, _ = self.get_date()
        X = X.reset_index(drop=True)
        prev_index = X[X["datetime"] == today_str].index[0] - 1
        X = X.iloc[[prev_index]]
        
        X.drop('datetime', axis=1, inplace=True)
        
        self.train_model()
        pred_direction_proba = self.clf_model.predict_proba(X)[0][1]

        return pred_direction_proba

    def position_sizing(self):
        cash = self.cash
        last_price = self.get_last_price(self.symbol)
        if last_price is None:
            raise ValueError(f"Last price for {self.symbol} is None. Cannot calculate position sizing.")
        quantity = cash * self.cash_at_risk // last_price
        return cash, last_price, quantity

    def on_trading_iteration(self):
        
        self.trading_iteration += 1
        
        if broker.is_market_open():
            _, last_price, _ = self.position_sizing()
            print(f"Last Price: {last_price:.2f}")
            
            pred_direction_proba = self.get_signal()
            print(f"pred_direction_proba: {pred_direction_proba}")

            if (pred_direction_proba >= 0.50):
                if self.last_trade == "short":
                    self.sell_all()
                cash, last_price, quantity = self.position_sizing()
                
                if cash > last_price:
                    order = self.create_order(
                        self.symbol,
                        quantity,
                        "buy",
                        type="bracket",
                        take_profit_price=last_price * 1.15,
                        stop_loss_price=last_price * 0.95,
                    )
                    self.submit_order(order)
                    self.last_trade = "long"

            if (pred_direction_proba < 0.50): 
                if self.last_trade == "long":
                    self.sell_all()
                cash, last_price, quantity = self.position_sizing()
                
                # if cash > last_price:
                #     order = self.create_order(
                #         self.symbol,
                #         quantity,
                #         "sell_short",
                #         type="bracket",
                #         take_profit_price=last_price * 0.90,
                #         stop_loss_price=last_price * 1.05,
                #     )
                #     self.submit_order(order)
                #     self.last_trade = "short"
        else:
            print("market is closed.")

# Backtesting

In [ ]:
# !pip install --upgrade lumibot

In [ ]:
# Pick the dates that you want to start and end your backtest
# and the allocated budget
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 5, 12)

symbol = 'SPY'
budget = 1000

# Run the backtest
trader = Trader(backtest=True)
data_source = YahooDataBacktesting(
    datetime_start=start_date,
    datetime_end=end_date,
)
broker = BacktestingBroker(data_source)
strat = MyStrategy(
    symbol=symbol,
    clf_model=clf_model,
    broker=broker,
    budget=budget
)

trader.add_strategy(strat)
trader.run_all()

# Paper Trading

In [29]:
ALPACA_CONFIG = {
    "API_KEY": api_key_alpaca,
    "API_SECRET": api_secret_alpaca,
    "PAPER": True
}

symbol = 'SPY'

alpaca = Alpaca(ALPACA_CONFIG, connect_stream=False)
strategy = src.trading_bot.MLStrategy(
    symbol=symbol,
    clf_model=clf_model,
    reg_model=reg_model,
    broker=alpaca
)
trader = Trader()
trader.add_strategy(strategy)
trader.run_all()

2024-05-13 22:42:15,547: asyncio: INFO: [MLStrategy] Executing the initialize lifecycle method
2024-05-13 22:42:17,307: asyncio: INFO: [MLStrategy] Sleeping until the market opens
2024-05-13 22:42:17,905: asyncio: INFO: [MLStrategy] Executing the before_market_opens lifecycle method
2024-05-13 22:42:18,577: asyncio: INFO: [MLStrategy] Sleeping until the market opens
2024-05-13 22:42:18,578: asyncio: INFO: [MLStrategy] Executing the before_starting_trading lifecycle method
2024-05-13 22:42:19,453: asyncio: WARNING: [MLStrategy] The strategy will run at 09:30:05 every day. If instead you want to start right now and run every 7 days then set force_start_immediately=True in the strategy's initialization.
2024-05-13 22:42:19,487: asyncio: INFO: [MLStrategy] Strategy will check in again at: 2024-05-14 09:30:05
